In [7]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
path = stockdata_csv_path = Path("Resources/NFLX.csv")
data = pd.read_csv(path)

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Handle missing values
data = data.dropna()


In [8]:
# Create lagged features
data['Lag1_Close'] = data['Close'].shift(1)
data['Lag2_Close'] = data['Close'].shift(2)

# Create additional features (e.g., volatility, daily return)
data['Volatility'] = (data['High'] - data['Low']) / data['Open']
data['Daily_Return'] = data['Close'].pct_change()

# Create target variables
data['Target_1M'] = data['Close'].shift(-21)

# Drop rows with NaN values created by shifting
data = data.dropna()


In [9]:
# Define features and target
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Lag1_Close', 'Lag2_Close', 'Volatility', 'Daily_Return']
target = 'Target_1M'

X = data[features]
y = data[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [10]:
# Build the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with epochs
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), 
                    callbacks=[EarlyStopping(patience=3)])


Epoch 1/50


c:\Users\caila\Desktop\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 46654.9961 - val_loss: 4254.0933
Epoch 2/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3240.6504 - val_loss: 1749.4900
Epoch 3/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2423.4780 - val_loss: 1430.4159
Epoch 4/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1920.9432 - val_loss: 1310.6351
Epoch 5/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1658.0465 - val_loss: 971.1737
Epoch 6/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1659.0532 - val_loss: 766.9868
Epoch 7/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1506.5094 - val_loss: 700.2137
Epoch 8/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1667.1193 - val_loss: 671.7071
Epoch 9/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1373.2988 - val_loss: 698.0754
Epoch 10/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1395.6998 - val_loss: 640.4968
Epoch 11/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1356.7495 - val_loss: 634.0941
Epo

In [11]:

# Make predictions
pred_1m = model.predict(X_test_scaled)

# Evaluate the model
mse_1m = mean_squared_error(y_test, pred_1m)
r2_1m = r2_score(y_test, pred_1m)

print(f'1-Month Prediction - MSE: {mse_1m}, R2: {r2_1m}')

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1-Month Prediction - MSE: 678.6707804053796, R2: 0.9793321864876962
